## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откроим файл с данными и изучитим общую информацию. 

In [1]:
import pandas as pd
#загрузим таблицу и получем информацию о ней
data_statistics = pd.read_csv('/datasets/data.csv')
print(data_statistics.head(10))
data_statistics.info()



   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

### В таблице имеется 12 столбцов и 21525 наблюдений, в разных столбцах разное количество значенний non-null - следовательно имеются пропущенные значения, разные типы данных. При просмотре данных таблицы можно заметить, что days_employed(стаж работы) имеет как положительные так и отрицательные значения, education(уровень образования) значения прописаны в разных регистрах, purpose - возможно написание одной и той же цели разными словосочетаниями.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
#в столбце "children" имеются отрицательные значения, приведем все значения к абсолютному
data_statistics['children'] = data_statistics['children'].abs()


In [3]:
#проверяем наличие пропусков в столбцах
print(data_statistics.isnull().sum())
#так как в столбце содержащим информацию о общим трудовом стаже в днях данные вероятно испорчены и приниматся в расчет
# дальнешем не будут заменим пропуски на нулевые значения.
data_statistics['days_employed'] = data_statistics['days_employed'].fillna(0)
#данные в столбце с информацией ежемесечного досода имеют ценность и будут расматриваться в дальнешем анализе
#найдем характерные значения в столбцах с пропусками сгрупироваными по типу занетости для более точного определения пропущенных значений
print(data_statistics.groupby('income_type')['total_income'].mean())

#заменим отсутствующие значения на характерные с учетом групп занятости
data_statistics['total_income'] = data_statistics['total_income'].fillna(data_statistics.groupby('income_type')
                                                                           ['total_income'].transform('mean'))

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.461462
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: total_income, dtype: float64


### Были обнаружены пропуски в столбцах days_employed - общий трудовой стаж и total_incometotal_income - ежемесечный доход.  При расмотрении  столбца days_employed можно заметить, что данные иеют труднообъснимые значения (часть отрицательные, многие имеют очень большие значения). Можно предположить, что данные в столбце испорченны и немогут приниматся к учету, вседствии этого можно заменить пропущенные в нем значения на нулевые, т.к. в дальнешем данный столбец не будем использовать при иследовании.  Напротив в столбце  total_incometotal_income содержатся ценные значения которые в дальнейшем будут использованы для анализа. Доходы заемщиков можно группировать по типу занятости  "income_type" и высчитать характерные значения для этих групп. Затем заменяем пропуски на характерный значения для этих столбцов в зависимости от типа занятости.

### При просмотре таблица можно заметить, что данные пропуски имеются у одних и тех же заемщиков. Пропуски могли возникнуть из-за не предоставлении заемщиком необходимой информации в полном объеме - отсутствия справки о ежемесечном доходе и трудовом стаже (песионер, учащийся, официально нетрудоустроенный).


### Замена типа данных

In [4]:
#проверим подсчет суммы в проблемных столбцах для исключения наличия в них строковых значений
print(data_statistics['days_employed'].sum())
print(data_statistics['total_income'].sum())
#в столбцах вешшественное число заменим на целое число
data_statistics['days_employed'] = data_statistics['days_employed'].astype('int')
data_statistics['total_income'] = data_statistics['total_income'].astype('int')
#проверим правильность замены
data_statistics.info()


1220012776.2471714
3603197086.325668
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Имеем столбцы: 2 вещественное число, 5 целое число, 5 стороковые. Заменим вещественное на целое float64 -> int64. есть два способа замены типа данных 1) to_numeric() если имеем некорректные значения в столбце и 2) .astype('int') для преобразования из вещестненного в целочисленный. Воспользовался вторым , так как сумма по выбранным столбцам подсчитывается корректно

### Обработка дубликатов

In [5]:
#проверим наличие дубликатов
print(data_statistics.duplicated().sum()) 

#приведем весь текст к нижнему регистру
data_statistics['education'] = data_statistics['education'].str.lower()
#удалим дубликаты
data_statistics = data_statistics.drop_duplicates().reset_index(drop = True)

#проверим наличие дубликатов после выполнения удаления 
print(data_statistics.duplicated().sum())


54
0


### применем к таблице метод .duplicated() для оценки наличия дубликатов из-за использования разного регистра при внесении данных. Видим написание разными регистрами -применим метод .str.lower() для приведения к нижнему регистру и метод .drop_duplicates() для удаления дубликатов. проверим -  дубликаты удаленны

### Лемматизация

In [6]:
#импортируем библиотеку pymystem3
from pymystem3 import Mystem
m = Mystem()
#создадим функцию для лемматизации
def purpose_lemmatize(text):
    lemmas = m.lemmatize(text) 
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'недвижимость'in lemmas:
        return 'недвижимость'
    if 'жилье' in lemmas:
        return 'недвижимость'
    if 'автомобиль'in lemmas:
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
#применим функцию к столбцу таблицы
data_statistics['purpose_lemmatize'] = data_statistics['purpose'].apply(purpose_lemmatize)                                                                                      
#print(data_statistics['purpose_lemmatize'])

### в столбце 'purpose' прописаны цели кредитования. Предпологается, что сходные цели прописаны в разных вариациях. Для последующей категоризации проведем леммитизацию столбца. Так как обращение будет проводится к одному столбцу из таблицы применим метод data['column'].apply(metod), перед этим импортируем библиотеку pymystem3 и создаем функцию. В функцию включим проверку на соответствие основным целям кредитования: автомобиль, свадьба, недвижимость (объединив с жильем), образования и поместим в таблицу.

### Категоризация данных

In [7]:
#создадим словари для иследования
data_children = data_statistics[['children','debt']]
data_family_status = data_statistics[['family_status','debt']]
data_total_income =data_statistics[['total_income','debt']]
data_purpose = data_statistics[['purpose_lemmatize','debt']]


### Производим категоризацию данных. На этом этапе создаем словари для анализа данных таблицы:
1.data_children - для иследования зависимости возврата кредита и наличия детей у заемщика;
2.data_family_status - для иследования зависимости возврата кредита и семейного положения заемщика;
3.data_total_income - для иследования зависимости возврата кредита и уровня дохода заемщика;
4.data_purpose - для иследования зависимости возврата кредита и цели кредита.,
Категоризацию проводим исходя из предположений, что выбранные данные могут повлиять на финансовое положение заемщика. 


### Шаг 3. Ответьтим на вопросы

### - Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [8]:
#создадим функцию для иследования зависимости наличия детей и наличия задолжности по кредиту 
def children_debt(row):
    debt = row['debt']
    if debt == 0:
        return 'no_debt'
    if debt == 1:
        return 'has_debt'

def children_name(row):
    children = row['children']
    if children == 0:
        return 'заемщик не имеющий детей'
    if children > 0:
        return 'заемщик с детьми'
#применим функцию к словарю
try:
    data_children['info'] = data_children.apply(children_debt, axis=1)
    data_children['name'] = data_children.apply(children_name, axis=1)
    data_children_pivot = data_children.pivot_table(index = 'name', columns = 'info',
                                                values = 'debt', aggfunc = 'count')
except:
    print('функция не работает')
    # получим соотношение в зависимости от наличия задолжности по группам
data_children_pivot['ratio'] = data_children_pivot['has_debt'] / data_children_pivot['no_debt']
# посмотрим результат
print(data_children_pivot)                                          



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


info                      has_debt  no_debt     ratio
name                                                 
заемщик не имеющий детей      1063    13028  0.081593
заемщик с детьми               678     6685  0.101421


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### согласно проведенному иследованию можно увидеть, что соотношение бездетных заемщиков имевших задолжности к  бездетным не имевшим задолжность составляет около 0,08 , а такое же соотношение среди имеющих детей составляет около 0,1 , что показавает различное соотношение по двум группам заемщиков и позволяет сделать вывод некотором уровне влияния  наличия детей у заемщика на появление задолжности по возврату кредита и предположить, что выдача кредитов заемщикам не имеющих детей предполагает несколько меньше  рисков по возникновению задолжности по кредиту  .


### - Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [9]:
#создадим функцию для иследования зависимости семейного положения и наличия задолжности по кредиту
def family_debt(row):
    debt = row['debt']
    if debt == 0:
        return 'no_debt'
    if debt == 1:
        return 'has_debt' 
            
data_family_status['info'] = data_family_status.apply(family_debt, axis=1)


data_family_status_pivot = data_family_status.pivot_table(index = 'family_status', columns = 'info',
                                                values = 'debt', aggfunc = 'count')
# получим соотношение в зависимости от наличия задолжности по группам
data_family_status_pivot['ratio'] = data_family_status_pivot['has_debt'] / data_family_status_pivot['no_debt']
##выведем на печать результат
print(data_family_status_pivot)    

info                   has_debt  no_debt     ratio
family_status                                     
Не женат / не замужем       274     2536  0.108044
в разводе                    85     1110  0.076577
вдовец / вдова               63      896  0.070312
гражданский брак            388     3763  0.103109
женат / замужем             931    11408  0.081609


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


### согласно проведенному иследованию можно увидеть, что соотношение  заемщиков    имевших задолжности к заемщикам той же группы не имевшим задолжность имеет яроко выраженую зависимость от семейного состояния заемщика. Так наименьшую зависимост 0,07 показала групп"вдовец/вдова", несколько большую зависимость имеют группы "в разводе" и "женат/замужем" около 0,076 и 0,08 соответствено, а наибольшую зависимость имеют группы"гражданский брак" и "не женат/не замужем"  около 0,10 и 0,11 соответственно.

### - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:

#установим границы максимальных, средних и минимальных уровней дохода
tall = (data_statistics['total_income'].max() + data_statistics['total_income'].mean()) / 2
low = (data_statistics['total_income'].min() + data_statistics['total_income'].mean()) / 2
print(tall, low)
#условно разделим данные на три группы по средним значениям между max, mean и min:
#с высоким доходом - свыше 1216512.9105761163; средним от 94044.41057611634 до 1216512.9105761163 вкючительно и 
#низким от 0 до 94044.41057611634 вкючительно
#создадим функцию для иследования зависимости уровня дохода и наличия задолжности по кредиту
def total_debt(row):
    debt = row['debt']
    if debt == 0:
        return 'no_debt'
    if debt == 1:
        return 'has_debt'

def total_name(row):
    total_status = row['total_income']
    if total_status <= low:
        return 'заемщик с низким уровнем доходов'
    if total_status > low:
        if total_status <= tall:
            return 'заемщик со средним уровнем доходов'
    if total_status > tall:
        return 'заемщик с высоким уровнем доходов'
#применим функцию к словарю
data_total_income['info'] = data_total_income.apply(total_debt, axis=1)
data_total_income['name'] = data_total_income.apply(total_name, axis=1)
data_total_income_status_pivot = data_total_income.pivot_table(index = 'name', columns = 'info',
                                                values = 'debt', aggfunc = 'count')
# получим соотношение в зависимости от наличия задолжности по группам
data_total_income_status_pivot['ratio'] = data_total_income_status_pivot['has_debt'] / data_total_income_status_pivot['no_debt']
##выведем на печать результат
print(data_total_income_status_pivot)  

# посмотрим соотношение в зависимости от уровня доходов


1216517.7918569962 94049.29185699637


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


info                                has_debt  no_debt     ratio
name                                                           
заемщик с высоким уровнем доходов          1       12  0.083333
заемщик с низким уровнем доходов         298     3473  0.085805
заемщик со средним уровнем доходов      1442    16228  0.088859


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### согласно проведенному иследованию можно увидеть, что соотношение  заемщиков  имевших задолжности к заемщикам той же группы не имевшим задолжность составляет: по группам доходов низкий около 0,085 , средний около 0,089 , высокий около 0,083 что показавает схожее соотношение по всем группам заемщиков и позволяет сделать вывод о очень крайне низком уровне влияния уровня доходов заемщика на появление задолжности по возврату кредита с некоторой долей вероятности большего уровня задолжности по кредиту у заемщиков со средним уровнем дохода. 

### - Как разные цели кредита влияют на его возврат в срок?

In [12]:
#просмотрим цели кредита указаные в таблице 
#print(data_statistics['purpose'].value_counts())
#можно выделить основные цели кредита: свадьба, недвижемость, жилье, автомобиль, образование.
#проведем иследование по этим группам целей кредита и воспользуемся раннее
#подготовленным лемматизированным столбцом
#создадим функцию для иследования зависимости целей кредита и наличия задожности по кредиту
def purpose_debt(row):
    debt = row['debt']
    if debt == 0:
        return 'no_debt'
    if debt == 1:
        return 'has_debt'
    
def purpose_name(row):
    purpose = row['purpose_lemmatize']
    if purpose == 'свадьба':
        return 'цель кредита на проведение свадьбы'
    if purpose == 'недвижимость':
        return 'цель кредита на приобретение недвижимости'
    if purpose == 'автомобиль':
        return 'цель кредита на приобретение автомобиля'
    if purpose == 'образование':
        return 'цель кредита на получение образования'
   
#применим функцию к словарю
data_purpose['info'] = data_purpose.apply(purpose_debt, axis=1)
data_purpose['name'] = data_purpose.apply(purpose_name, axis=1)
data_purpose_pivot = data_purpose.pivot_table(index = 'name', columns = 'info',
                                                values = 'debt', aggfunc = 'count')
# получим соотношение в зависимости от наличия задолжности по группам
data_purpose_pivot['ratio'] = data_purpose_pivot['has_debt'] / data_purpose_pivot['no_debt']
##выведем на печать результат
data_purpose_pivot

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


info,has_debt,no_debt,ratio
name,,,
цель кредита на получение образования,370,3643,0.101565
цель кредита на приобретение автомобиля,403,3903,0.103254
цель кредита на приобретение недвижимости,782,10029,0.077974
цель кредита на проведение свадьбы,186,2138,0.086997


### согласно проведеному иследованию можно увиденть, что соотношение заемщиков имевшим задолжность по кредиту к заемщикам не имевшим задолжность по группам  состовляет: автомобиль около 0,10, образование составляет около 0,10, свадьба около 0,09 и недвижимость около 0,08, что показывает наличие различия в в уровне зависимости возникновения задолжности от цели кредитования. Так наиболее сильно риск появление задолжности проявляется в группах с целью кредитования автомобиль и образование, наименьшая  вероятность появления задолжности у цели недвижемость  

### Шаг 4. Общий вывод 

### Проведя иследования зависимости между возвратом в срок кредита и различными факторами относящимися к заемщику можно сделать вывод:
####    Различные факторы имеют разную степень влияния на появление задолжности по кредиту, так анализ уровня доходов показывает крайне низкую зависимость от уровня доходов, а анализ показателей наличия детей, целей кредитрования, семейного положения показывает более значительную зависимость от иследуемых признаков.
    
####    Проведенный анализ указывает, что возникновению задолжности скорее способствует не уровень доходов, а уровень расходов заемщика. Так различие в уровне доходов показывают примерно одинаковую вероятность возникновения задолжности у заемщиков с высоким и низким уровнем доходов с небольшим улучшение ситуации в сторону заемщиков со средним доходом: высокий-0,83 средний-0,88 низкий-0,85. Другие факторы можно охарактеризовать зависимостью от дополнительных факторов влияющих на уровень расходов Так заемщики имеющие детей вероятно расходуют дополнительную часть дохода на детей повышая тем самым риск возникновения задолжности по кредиту: соотношение бездетных заемщиков имевших задолжности к бездетным не имевшим задолжность составляет около 0,08 , а такое же соотношение среди имеющих детей составляет около 0,1. По цели кредитования показатель "на приобретения жнедвижимости" проявил себя наилучшим образом (индекс- около 0,08)вероятно, что до приобретения собственного жилья заемщики могли тратить часть дохода на аренду жилья и с приобретением собственного часть дохода высвобождается позволяя избежать возникновения задожности. Напротив цель "получение образования" (индекс - около 0,10) позволяет предположить, что на время получения образования доходная часть бюджета заемщика снизится из-за сложности совмещения учебы и работы. Также заемщики приобревшие автомобиль получают дополнительные расходы по содержания автомобиля (индекс - около 0,10). Анализ по показателю семейное положение показал, что заемщики ранее состоявшие в официальном браке показали наилучшее показатели 0,07 показала групп"вдовец/вдова", несколько большую зависимость имеют группы "в разводе" и состоящие в настоящее время в браке "женат/замужем" около 0,076 и 0,08 соответствено. Напротив наибольшую зависимость имеют группы"гражданский брак" и "не женат/не замужем" около 0,10 и 0,11 соответственно. Возможно это обусловлено тем, что заемщики ранее бывшие в браке и состоящие в нем более ответствено подходят к финансовой состовляющей их жизни.
    
 ####    На основании полученных данных можно создать образ идеального заемщика для выдачи кредита - это будет являтся заемщик не имеющий детей, ранее бывший или состоящий на настоящий момент в браке с высоким уровнем доходов с целью кредитования на "недвижемость", наименее привлекательным заемщиком будет выглядеть заемщик с детьми, состоящий не состоящим в официальном браке имеющий средний уровень доходов с целью кредитования "образование" или "автомобиль".
